In [1]:
import anthropic

In [2]:
client = anthropic.Anthropic(api_key="your-api-key")

In [4]:
from IPython.display import Markdown, display
from anthropic import HUMAN_PROMPT, AI_PROMPT

In [5]:
Prompt = "What ia an onion?"
message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": Prompt}
    ]
)
Markdown(message.content[0].text)

An onion is a vegetable that belongs to the Allium genus, which also includes garlic, shallots, leeks, and chives. Here are some key characteristics of onions:

1. Appearance: Onions are round or oval-shaped bulbs with papery skin that can be white, yellow, red, or purple.

2. Layers: Onions consist of multiple layers of fleshy scales, which are arranged concentrically around the core.

3. Flavor: They have a distinctive, pungent flavor and aroma due to sulfur-containing compounds released when the onion is cut or crushed.

4. Culinary uses: Onions are widely used in cooking as a base for various dishes, such as soups, stews, sauces, and stir-fries. They can be eaten raw, sautéed, caramelized, or roasted.

5. Health benefits: Onions are low in calories and rich in vitamins (C and B6), minerals (potassium and folate), and antioxidants (quercetin). They may offer potential health benefits, such as reducing the risk of certain cancers and heart disease.

6. Tear-inducing properties: Cutting onions releases a volatile compound called syn-propanethial-S-oxide, which irritates the eyes and causes tears.

Onions are a staple ingredient in many cuisines worldwide and are prized for their versatility and flavor-enhancing properties.